# Generate Search Summary
This notebook retrieves different subsets of the search and dumps them into a spreadsheet

In [1]:
%matplotlib inline
from matplotlib import pyplot as plt
from moldesign.store.mongo import MoleculePropertyDB
from moldesign.store.models import MoleculeData
from moldesign.store.recipes import redox_recipes
from rdkit.Chem import Descriptors
from rdkit import Chem
from pathlib import Path
from tqdm import tqdm
from datetime import datetime
from scipy.interpolate import interp1d
import pandas as pd
import numpy as np
import json
import yaml

The history saving thread hit an unexpected error (DatabaseError('database disk image is malformed')).History will not be written to the database.


RDKit WARNING: [15:12:37] Enabling RDKit 2019.09.3 jupyter extensions
[15:12:37] Enabling RDKit 2019.09.3 jupyter extensions


Define the search parameters

In [2]:
subset = 'ZINC15'  # Allowed source of the molecules
neutral = True  # Whether to get only charge-balanced molecules

Gather which properties to look up.

In [3]:
levels = [level.name for level in redox_recipes] 
solvation_energy = {'small': 'data.small_basis.neutral.solvation_energy.neutral.acetntrl.small_basis',
                    'normal': 'data.small_basis.neutral.solvation_energy.neutral.acetntrl.normal_basis',
                    'diffuse': 'data.small_basis.neutral.solvation_energy.neutral.acetntrl.diffuse_basis'}

## Query the current state of the search from MongoDB
Gather only the molecules in the target subset and return the accires at all available levels

Query the database to get the desired results

In [4]:
mongo = MoleculePropertyDB.from_connection_info(port=27855)

In [5]:
hits = []
for hit in tqdm(mongo.collection.find({'subsets': {'$in': [subset]}})):
    hits.append(MoleculeData.parse_obj(hit))

0it [00:30, ?it/s]


ServerSelectionTimeoutError: localhost:27855: [Errno 111] Connection refused, Timeout: 30s, Topology Description: <TopologyDescription id: 628fd1262d6752f6ae737b60, topology_type: Single, servers: [<ServerDescription ('localhost', 27855) server_type: Unknown, rtt: None, error=AutoReconnect('localhost:27855: [Errno 111] Connection refused')>]>

In [ ]:
results = []
for hit in tqdm(hits):
    # Get the basic information
    result = {'smiles': hit.identifier['smiles'],
              'inchi': hit.identifier['inchi'],
              'record': hit,
              'molwt': Descriptors.MolWt(hit.mol),
              'charge': Chem.GetFormalCharge(hit.mol),
             'n_atoms': hit.mol.GetNumHeavyAtoms()}
    
    # Store the reduction potentials
    for label, data in zip(['ip', 'ea'], [hit.oxidation_potential, hit.reduction_potential]):
        for level in levels:
            result.update({f'{label}_' + level: data.get(level)})
    
    # Get the solvation
    for label, key in solvation_energy.items():
        try:
            result[f'solv_eng_{label}'] = hit.get_data(key)
        except KeyError:
            continue
    
    results.append(result)
results = pd.DataFrame(results)

Get only the nuetral molecules, if desired

In [ ]:
if neutral:
    results.query('charge == 0', inplace=True)

Write out a subset of the data frame to illustrate its content

In [ ]:
results.sort_values('ip_nob-acn-smb-geom', ascending=False)[['smiles', 'molwt', 'ip_nob-acn-smb-geom', 'solv_eng_normal']]

## Plot the Pareto Surface
What about our tradeoff betweeen solvation energy and EA

In [ ]:
def get_dominating_solutions(results, redox_key = 'ea_dfb-acn-smb-geom', solv_key = 'solv_eng_diffuse', mols_to_skip = ()): 
    """Get the results with the best solvation energy for a certain redox potential
    
    Args:
        results: Results array to process
        redox_key: Name of the redox column
        solv_key: Name fo the solvation column
        mols_to_skip
    Returns:
        - Version of the results array with the pareto surface identified and distances from the surface collected
    """
    
    # Get a copy of the array
    results = results.copy()
    results = results[~results[[redox_key, solv_key]].isnull().any(axis=1)]
    
    # Remove all columns except those we are interested in
    results = results[['inchi', 'smiles', 'molwt', 'charge', redox_key, solv_key]]
    
    # Get rid of molecules in the skip list
    results = results[results.smiles.apply(lambda x: x not in mols_to_skip)]
    
    # Put a placeholder for whether something is on the surface
    #  and by how much it is off
    results['is_pareto'] = False
    results['redox_dist'] = 0
    results['solv_dist'] = 0

    # Get the dominating solutions
    pareto = []
    output = results
    results.sort_values(redox_key, inplace=True, ascending=redox_key.startswith('ea_'))
    while len(results) > 0:
        # Get the best value and remove it from the list
        new_best = results.iloc[0]
        
        # Mark the best as pareto optimal
        output.loc[results.index[0], 'is_pareto'] = True
        
        # Find all entries that also have a worse solvation
        dominated = results[solv_key] > new_best[solv_key]
        
        # Store the "dominated distance" 
        output.loc[results.index, 'redox_dist'] = np.abs(results[redox_key] - new_best[redox_key])
        output.loc[results.index, 'solv_dist'] = np.abs(new_best[solv_key] - results[solv_key])
        
        # Mark it as dominated
        results = results[~dominated]
        results = results.iloc[1:]
        
    # Sort by the distance to the dominating redox solution
    output.sort_values(['redox_dist', 'solv_dist'], ascending=True, inplace=True)
                          
    return output
pareto_front = get_dominating_solutions(results)

In [ ]:
ea_results = get_dominating_solutions(results)

In [ ]:
ea_results

In [ ]:
fig, ax = plt.subplots(figsize=(3.5, 2.5))

pareto_front = ea_results.query('is_pareto')
ax.scatter(ea_results['ea_dfb-acn-smb-geom'], ea_results['solv_eng_diffuse'])
ax.scatter(pareto_front['ea_dfb-acn-smb-geom'], pareto_front['solv_eng_diffuse'], s=35, marker='s', color='crimson')
ax.step(pareto_front['ea_dfb-acn-smb-geom'], pareto_front['solv_eng_diffuse'], 'k--', where='post')

ax.set_xlabel('EA (V)')
ax.set_ylabel('$G_{solv}$ (kcal/mol)')

fig.tight_layout()
fig.savefig('pthalimide-pareto.png', dpi=320)

In [ ]:
pareto_front

In [ ]:
ip_results = get_dominating_solutions(results, 'ip_nob-acn-smb-geom', 'solv_eng_normal')#, mols_to_skip=('F[N+](F)(F)F', 'N#Cc1oncc1C=O'))

In [ ]:
ip_results

In [ ]:
fig, ax = plt.subplots(figsize=(3.5, 2.5))

pareto_front = ip_results.query('is_pareto')
ax.scatter(ip_results['ip_nob-acn-smb-geom'], ip_results['solv_eng_normal'])
ax.scatter(pareto_front['ip_nob-acn-smb-geom'], pareto_front['solv_eng_normal'], s=35, marker='s', color='crimson')
ax.step(pareto_front['ip_nob-acn-smb-geom'], pareto_front['solv_eng_normal'], 'k--', where='post')

ax.set_xlabel('IP (V)')
ax.set_ylabel('$G_{solv}$ (kcal/mol)')

fig.tight_layout()

In [ ]:
pareto_front

## Save them to disk
Let's make a memorable filename and dump accordingly

In [ ]:
filename = datetime.now().strftime('%y%m%d')  # Start with the date

In [ ]:
if neutral:   # Mark whether we only output the neutrals
    filename += "-neutral_only"

In [ ]:
if subset:
    filename += f'-subset_{subset}'

In [ ]:
print(f'Filename postfix: {filename}')

Save the data to a collection folder

In [ ]:
output_folder = Path('results')
output_folder.mkdir(exist_ok=True)

In [ ]:
ea_results.to_csv(output_folder / f'anolytes-{filename}.csv', index=False)

In [ ]:
ip_results.to_csv(output_folder / f'catholytes-{filename}.csv', index=False)

## Make a tabular form
Render the data into HTML

In [ ]:
results.